## Expirement

KMeans clustering using MASE joint embedding (FA_R2, MD_R2, IS_MDF)
 
##### Determine number clusters per bundle based on population data

**NOTE `662551` not in `HCP_1200`**

```
s3://profile-hcp-west/hcp_reliability/single_shell/hcp_1200_afq/
s3://profile-hcp-west/hcp_reliability/single_shell/hcp_1200_afq_CSD/
```

In [1]:
experiment_names = ['MASE_FA_and_MD_Sklearn_KMeans']

subjects = [
    '103818', '105923', '111312', '114823', '115320',
    '122317', '125525', '130518', '135528', '137128',
    '139839', '143325', '144226', '146129', '149337',
    '149741', '151526', '158035', '169343', '172332',
    '175439', '177746', '185442', '187547', '192439',
    '194140', '195041', '200109', '200614', '204521',
    '250427', '287248', '341834', '433839', '562345',
    '599671', '601127', '627549', '660951', # '662551', 
    '783462', '859671', '861456', '877168', '917255'
]

# session_names = ['HCP_1200']
session_names = ['HCP_1200', 'HCP_Retest']

# bundle_names = ['SLF_L']
bundle_names = ['SLF_L', 'SLF_R']
# bundle_names = ['SLF_L', 'SLF_R', 'ARC_L', 'ARC_R', 'CST_L', 'CST_R']
# bundle_names = [
#     'ATR_L', 'ATR_R',
#     'CGC_L', 'CGC_R',
#     'CST_L', 'CST_R',
#     'IFO_L', 'IFO_R',
#     'ILF_L', 'ILF_R',
#     'SLF_L', 'SLF_R',
#     'ARC_L', 'ARC_R',
#     'UNC_L', 'UNC_R',
#     'FA', 'FP'
# ]

# range_n_clusters = range(2,10)
range_n_clusters = [2, 3, 4] # choosing minimal bundles and maximum to get silhouette

In [2]:
import itertools
# args = list(itertools.product(subjects, session_names, bundle_names))
args = list(itertools.product(experiment_names, subjects, session_names, bundle_names, range_n_clusters))
args

[('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_L', 2),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_L', 3),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_L', 4),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_R', 2),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_R', 3),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_1200', 'SLF_R', 4),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_L', 2),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_L', 3),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_L', 4),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_R', 2),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_R', 3),
 ('MASE_FA_and_MD_Sklearn_KMeans', '103818', 'HCP_Retest', 'SLF_R', 4),
 ('MASE_FA_and_MD_Sklearn_KMeans', '105923', 'HCP_1200', 'SLF_L', 2),
 ('MASE_FA_and_MD_Sklearn_KMeans', '105923', 'HCP_1200', 'SLF_L', 3),
 ('MASE_

useful for when individual jobs fail

all-in-one: `subbundle3`, `subbundle4`, and `subbundle5`

**Multiple Adjacency Spectral Emdedding (MASE)** 

- UNWARPED **Fractional Anisotropy Coefficient of Determination (FA R2)** 

- UNWARPED **Mean Diffusivity Coefficient of Determination (MD R2)** 

- **Inverse Scaled Minimum average Direct-Flip (IS MDF) distance**

In [3]:
def subbundle(experiment_name, subject, session, bundle_name, n_clusters, clean_bundles=True):
    """
    Run clustering for K=`n_clusters` on `subject`, `session`, `bundle_name` 
    using `clean_bundles` CSD tractography
    
    Features:
    - Tissue - Fractional Anisotropy Coefficient of Determination (FA R^2) 
    - Tissue - Mean Diffusivity Coefficient of Determination (MD R^2)
    - Distance - Inverse Scaled Minimum average Direct-Flip distance (IS_MDF)
    
    Generates:
    - Joint graph adjacncy spectral embeddings using `MASE`
    - Clusters using both `KMeansCluster`
    """
    import logging
    import time
    import s3fs
    import glob
    import numpy as np
    import pandas as pd
    from dipy.io.streamline import load_tractogram, save_tractogram
    from dipy.io.stateful_tractogram import StatefulTractogram
    from dipy.io.utils import create_nifti_header, get_reference_info
    from dipy.tracking.streamline import set_number_of_points, values_from_volume, bundles_distances_mdf
    import dipy.tracking.utils as dtu
    import nibabel as nib
    from graspologic.embed import MultipleASE as MASE
#     from graspologic.cluster import KMeansCluster, GaussianCluster
#     from graspologic.plot import pairplot
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    from seaborn import lineplot
    import matplotlib.pyplot as plt
    
    logger = logging.getLogger("s3fs")
    logger.setLevel(logging.DEBUG)
    
    def coeff_of_determination(data, model, axis=-1):
        """
         http://en.wikipedia.org/wiki/Coefficient_of_determination
                  _                                            _
                 |    sum of the squared residuals              |
        R^2 =    |1 - ---------------------------------------   | * 100
                 |_    sum of the squared mean-subtracted data _|
        """
        X = np.empty((data.shape[0], model.shape[0]))
        demeaned_data = data - np.mean(data, axis=axis)[...,np.newaxis]
        ss_tot = np.sum(demeaned_data **2, axis=axis)
        # Don't divide by 0:
        if np.all(ss_tot==0.0):
            X[:, :] = np.nan
            return X
        for ii in range(X.shape[0]):
            for jj in range(X.shape[1]):
                # There's no point in doing any of this: 
                if np.all(data[ii]==0.0) and np.all(model[ii]==0.0):
                    X[ii, jj] = np.nan
                else:
                    residuals = data[ii] - model[jj]
                    ss_err = np.sum(residuals ** 2, axis=axis)
                    X[ii, jj] = 1 - (ss_err/ss_tot[ii])
        return X

    def rrss(y, yhat):
        """
        Compute root residual sum of squares
        """
        residuals = y - yhat
        rss = np.dot(residuals.T, residuals)
        rrss = np.sqrt(rss)
        
        return rrss

    def relabel_clusters(cluster_labels):
        """
        Arrange cluster labels by number of streamlines
        """
        from_values = np.flip(np.argsort(np.bincount(cluster_labels))[-(np.unique(cluster_labels).size):])
        to_values = np.arange(from_values.size)

        d = dict(zip(from_values, to_values))

        new_cluster_labels = np.copy(cluster_labels)

        for k, v in d.items():
            new_cluster_labels[cluster_labels == k] = v

        return new_cluster_labels
        
    def density_map(tractogram):
        """
        Generate a density map for given tractogram
        """
        affine, vol_dims, voxel_sizes, voxel_order = get_reference_info(tractogram)
        tractogram_density = dtu.density_map(tractogram.streamlines, np.eye(4), vol_dims)
        tractogram_density = np.uint8(tractogram_density)
        nifti_header = create_nifti_header(affine, vol_dims, voxel_sizes)
        density_map_img = nib.Nifti1Image(tractogram_density, affine, nifti_header)
        
        return density_map_img
        
    def save_cluster_tractograms_and_density_maps(bundle_tractogram, model_prefix, cluster_labels):   
        """
        For each cluster save the tractogram and corresponding density map
        """
        for cluster_label in np.unique(cluster_labels):
            # tractogram
            f_name = f'{model_prefix}_cluster_{cluster_label}.trk'
            cluster_indicies = np.array(np.where(cluster_labels == cluster_label)[0])
            tg = StatefulTractogram.from_sft(bundle_tractogram.streamlines[cluster_indicies], bundle_tractogram)
            save_tractogram(tg, f_name, bbox_valid_check=False)
            print('saving cluster tractogram:', f_name, flush=True)
            
            # density map -- 8-bit unsigned int and gz
            f_name = f'{model_prefix}_cluster_{cluster_label}_density_map.nii.gz'
            tg.to_vox()
            nib.save(density_map(tg), f_name)
            print('saving cluster density map:', f_name, flush=True)
            
            # TODO save cluster afq_profile
            
    def cluster(clusterer, embedding, model_prefix):
        """
        Run `fit_predict`, relabel according to number of streamlines in each cluster, and 
        save classifications.
        
        The idx file contains a label from `0` to `n_clusters - 1` for each streamline in the
        tractogram.
        """
        idx = clusterer.fit_predict(embedding)
        # sort cluster by size, for convience, 
        # will need to resort by cross-subject similarity
        idx = relabel_clusters(idx)
        f_name = f'{model_prefix}_idx.npy'
        np.save(f_name, idx)
        print('saving:', f_name, flush=True)
        return idx
            
    def run_cluster_algos(tractogram, embedder_name, embedding, embedding_name):
        """
        Run given suite of clustering algorithms. Currently only using `KMeans`.
        
        Generate corresponding metadata and artifacts:
        - silhoutte scores
        - pair plot
        """
        # KMeans
        kmeans_model_prefix = f'{embedder_name}_kmeans_{embedding_name}'
        # grapsologic KMeansCluster with sci-kit so can force number of clusters
#         kmeans_clusterer = KMeansCluster(n_clusters)
        kmeans_clusterer = KMeans(n_clusters)
        kmeans_idx = cluster(kmeans_clusterer, embedding, kmeans_model_prefix)
        
        kmeans_info = pd.DataFrame(data={
            'subject': [subject], 
            'session': [session], 
            'bundle': [bundle_name], 
            'algorithm': ['kmeans'], 
            'embed dimensions': [embedding.shape], 
            'n_clusters' : [n_clusters],
        # grapsologic KMeansCluster with sci-kit so can force number of clusters
#             'max n_clusters' : [n_clusters],
#             'n_clusters selected': [kmeans_clusterer.n_clusters_], 
            'labels': [np.unique(kmeans_idx)], 
        # grapsologic KMeansCluster with sci-kit so can force number of clusters
#             'scores': [kmeans_clusterer.silhouette_]
            'scores': [silhouette_score(embedding, kmeans_idx)]
        })
        kmeans_info.to_pickle(f'{kmeans_model_prefix}_info.pkl')
        print('saving:', f'{kmeans_model_prefix}_info.pkl', flush=True)
        
        
        # TODO replace silhouette scores with sklearn
        # save silhouette scores so can use to determine n_clusters for population
#         print(f'{embedder_name} kmeans silhoutte\n', kmeans_clusterer.silhouette_)
#         np.save(f'{kmeans_model_prefix}_silhouette_scores.npy', kmeans_clusterer.silhouette_)
#         print('saving:', f'{kmeans_model_prefix}_silhouette_scores.npy')
        
#         lineplot(data=kmeans_clusterer.silhouette_).set(
#             title=f'{subject} {session} {bundle_name}\nkmeans n_clusters: {n_clusters} silhoutte scores',
#             xticks=range(len(range(2,n_clusters+1))),
#             xticklabels=range(2,n_clusters+1)
#         )
#         plt.savefig(f'{kmeans_model_prefix}_silhouette_scores.png')
#         print('saving:', f'{kmeans_model_prefix}_silhouette_scores.png')
        
        # TODO replace pairplot with sklearn
#         plt.clf()
#         pairplot(embedding, kmeans_idx, font_scale=0.55,
#                  title=f'{subject} {session} {bundle_name}\nkmeans n_clusters: {n_clusters}')
#         plt.savefig(f'{kmeans_model_prefix}_pairplot.png')
#         print('saving:', f'{kmeans_model_prefix}_pairplot.png')
        
        save_cluster_tractograms_and_density_maps(tractogram, kmeans_model_prefix, kmeans_idx)
        
    def run_embeddings(tractogram, features, feature_name):
        """
        multiple adjacency spectral embedding (mase)
        """
        print('mase', flush=True)
#         tic = time.perf_counter()

        # TODO specify embedding dimension D
        embedder = MASE()
        embedder_name = 'mase'
        embedding = embedder.fit_transform(features)
        print(embedder_name, feature_name, 'embedding dimension', embedding.shape, flush=True)
        run_cluster_algos(tractogram, embedder_name, embedding, feature_name)

#         toc = time.perf_counter()
#         print(f'mase {toc - tic:0.4f} seconds')

    print("begin", experiment_name, subject, session, bundle_name, n_clusters, flush=True)
    
    fs = s3fs.S3FileSystem()
    
    ### fractional anisotropy scalar file ###
    fa_scalar_filename = 'FA.nii.gz'
    print('loading FA scalar file:', fa_scalar_filename, flush=True)
#     tic = time.perf_counter()
    
    fs.get(
        (
            f'profile-hcp-west/hcp_reliability/single_shell/'
            f'{session.lower()}_afq_CSD/sub-{subject}/ses-01/'
            f'sub-{subject}_dwi_model-DTI_FA.nii.gz'
        ),
        f'{fa_scalar_filename}'
    )
    
    fa_scalar_data = nib.load(fa_scalar_filename).get_fdata()
#     toc = time.perf_counter()
#     print(f'scalar file: {toc - tic:0.4f} seconds')
    
    ### mean diffusivity scalar file ###
    md_scalar_filename = 'MD.nii.gz'
    print('loading scalar file: ', md_scalar_filename, flush=True)
#     tic = time.perf_counter()
    
    fs.get(
        (
            f'profile-hcp-west/hcp_reliability/single_shell/'
            f'{session.lower()}_afq_CSD/sub-{subject}/ses-01/'
            f'sub-{subject}_dwi_model-DTI_MD.nii.gz'
        ),
        f'{md_scalar_filename}'
    )
    
    md_scalar_data = nib.load(md_scalar_filename).get_fdata()
#     toc = time.perf_counter()
#     print(f'scalar file: {toc - tic:0.4f} seconds')
    
    ### single shell deterministic bundle tractography ###
    tractogram_filename = f'{bundle_name}.trk'
    print('loading tractogram:', tractogram_filename, flush=True)
#     tic = time.perf_counter()

    bundle_folder = 'bundles'
    
    if clean_bundles:
        bundle_folder = 'clean_' + bundle_folder
        
    fs.get(
        (
            f'profile-hcp-west/hcp_reliability/single_shell/'
            f'{session.lower()}_afq_CSD/sub-{subject}/ses-01/'
            f'{bundle_folder}/sub-{subject}_dwi_space-RASMM_model-CSD_desc-det-afq-{bundle_name}_tractography.trk'
        ),
        f'{tractogram_filename}'
    )
    
    tractogram = load_tractogram(tractogram_filename, 'same')
#     toc = time.perf_counter()
#     print(f'tractogram file: {toc - tic:0.4f} seconds')
    
    ### streamline profiles ###
    print('calculating streamline profiles', flush=True)
#     tic = time.perf_counter()
    n_points = 100
    
    fgarray = set_number_of_points(tractogram.streamlines, n_points)
    
    if len(fgarray) == 0:
        return
    
    # FA Values
    fa_values = np.array(values_from_volume(fa_scalar_data, fgarray, tractogram.affine))
    f_name = 'streamline_profile_fa.npy'
    np.save(f_name, fa_values)
    print('saving:', f_name, flush=True)
#     toc = time.perf_counter()

    print('fa values:', fa_values.shape, flush=True)
    
    # MD Values
    md_values = np.array(values_from_volume(md_scalar_data, fgarray, tractogram.affine))
    f_name = 'streamline_profile_md.npy'
    np.save(f_name, md_values)
    print('saving: ', f_name, flush=True)
#     toc = time.perf_counter()

    print('md values:', md_values.shape, flush=True)
    
#     print(f'streamline profile: {toc - tic:0.4f} seconds')
    
    ### Inverse Scaled MDF (Minimum average Direct-Flip) ###
    print('calculating mdf', flush=True)
#     tic = time.perf_counter()
    mdf = bundles_distances_mdf(fgarray, fgarray)
    
    # enforce symmetry
    mdf = (mdf + mdf.T) / 2
    
    # inverse scale
    is_mdf = (mdf.max() - mdf)
    is_mdf = is_mdf / is_mdf.max()

    f_name = 'adjacency_is_mdf.npy'
    np.save(f_name, is_mdf)
    print('saving:', f_name, flush=True)
#     toc = time.perf_counter()
    
    print('is_mdf:', is_mdf.shape, flush=True)
#     print(f'mdf {toc - tic:0.4f} seconds')
    
    ### streamline r2 ###
    print('calculating streamline r2', flush=True)
#     tic = time.perf_counter()
    
    # calculate FA R2
    fa_r2 = coeff_of_determination(fa_values, fa_values)
    
    # enforce symmetry
    fa_r2 += fa_r2.T
    fa_r2 = fa_r2/2
    
    # save file
    f_name = 'adjacency_fa_r2.npy'
    np.save(f_name, fa_r2)
    print('saving:', f_name, flush=True)
    
    print('adjacency_fa_r2:', fa_r2.shape, flush=True)
    
    # calculate MD R2
    md_r2 = coeff_of_determination(md_values, md_values)
    
    # enforce symmetry
    md_r2 += md_r2.T
    md_r2 = md_r2/2
    
    # save file
    f_name = 'adjacency_md_r2.npy'
    np.save(f_name, md_r2)
    print('saving: ', f_name, flush=True)
    
    print('adjacency_md_r2:', md_r2.shape, flush=True)
    
#     toc = time.perf_counter()
    
#     print(f'streamline r2: {toc - tic:0.4f} seconds')
    
    ### clustering ###
    
    fa_tissue = np.load('adjacency_fa_r2.npy')
    md_tissue = np.load('adjacency_md_r2.npy')
    distance = np.load('adjacency_is_mdf.npy')
    features = [fa_tissue, md_tissue, distance]
    feature_name = 'fa_r2_md_r2_is_mdf'
    run_embeddings(tractogram, features, feature_name)
    
    ### upload everything to s3 ###
    print(f'uploading to s3://hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/', flush=True)
    fs.put('*.npy', f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/')
    fs.put('*.nii.gz', f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/')
#     # some reason glob doesn't work with single file
    pkl_files = glob.glob('*.pkl')
    if len(pkl_files) == 1:
        pkl_file = pkl_files[0]
        fs.put(pkl_file, f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/{pkl_file}')
    else:
        fs.put('*.pkl', f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/')
#     fs.put('*.png', f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/')
    fs.put('*.trk', f'hcp-subbundle/{experiment_name}/{session}/{bundle_name}/{subject}/{n_clusters}/')
    
    print("end", experiment_name, subject, session, bundle_name, n_clusters, flush=True)

test locally before running on AWS

In [ ]:
import os
import glob

local_args = list(itertools.product(experiment_names, subjects[:5], session_names, bundle_names, range_n_clusters))
for (experiment_name, subject, session, bundle_name, n_clusters) in local_args:
    subbundle(experiment_name, subject, session, bundle_name, n_clusters)
    
    # delete inbetween otherwise uploading corrupted data
    for extensions in ["*.nii.gz", "*.trk", "*.npy", "*.pkl"]:
        for file in glob.glob(extensions):
            os.remove(file)

## AWS

Reconnect to existing knot

Reuse existing Docker Image

delete everything associate to the knot